# Data Availability

In [1]:
from huggingface_hub import HfApi
import pandas as pd
import itertools
import re
import requests
from bs4 import BeautifulSoup
import time
import json
import os

from tags import * # tags.py

/home/frabet/miniconda3/envs/hf-api-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
api = HfApi()

### LLMs

In [37]:
models = api.list_models(full=True)

In [4]:
model = itertools.islice(models, 0, 1000)
models_df = pd.DataFrame(model)
models_df.head(10)

,id,author,sha,created_at,last_modified,private,gated,disabled,downloads,downloads_all_time,...,pipeline_tag,mask_token,card_data,widget_data,model_index,config,transformers_info,siblings,spaces,safetensors
0,albert/albert-base-v1,albert,082438ba120d36b97b9288772a41144e941705b9,2022-03-02 23:29:04+00:00,2024-02-19 10:57:35+00:00,False,False,None,16157,None,...,fill-mask,None,None,None,None,None,None,"[{'rfilename': '.gitattributes', 'size': None,...",None,None
1,albert/albert-base-v2,albert,8e2f239c5f8a2c0f253781ca60135db913e5c80c,2022-03-02 23:29:04+00:00,2024-02-19 10:58:14+00:00,False,False,None,2462868,None,...,fill-mask,None,None,None,None,None,None,"[{'rfilename': '.gitattributes', 'size': None,...",None,None
2,albert/albert-large-v1,albert,94fd741fb5d6cb5bc578fc154837016c583bafef,2022-03-02 23:29:04+00:00,2024-02-19 10:58:26+00:00,False,False,None,2016,None,...,fill-mask,None,None,None,None,None,None,"[{'rfilename': '.gitattributes', 'size': None,...",None,None
3,albert/albert-large-v2,albert,dfed3a5ef4499fb3351c4ebbcf487375d1e942c8,2022-03-02 23:29:04+00:00,2024-02-19 10:58:48+00:00,False,False,None,12228,None,...,fill-mask,None,None,None,None,None,None,"[{'rfilename': '.gitattributes', 'size': None,...",None,None
4,albert/albert-xlarge-v1,albert,ed6f87d14403b3c459a458fa6aa9dc5c51c517c1,2022-03-02 23:29:04+00:00,2024-02-19 11:01:28+00:00,False,False,None,1424,None,...,fill-mask,None,None,None,None,None,None,"[{'rfilename': '.gitattributes', 'size': None,...",None,None
5,albert/albert-xlarge-v2,albert,4fd2c2aa9aeb305f87704a7e595be7bfffa3db88,2022-03-02 23:29:04+00:00,2024-04-10 09:57:46+00:00,False,False,None,3853,None,...,fill-mask,None,None,None,None,None,None,"[{'rfilename': '.gitattributes', 'size': None,...",None,None
6,albert/albert-xxlarge-v1,albert,43129068ee5f6a481c148daeac11cc593b8ff440,2022-03-02 23:29:04+00:00,2024-02-19 11:01:42+00:00,False,False,None,4894,None,...,fill-mask,None,None,None,None,None,None,"[{'rfilename': '.gitattributes', 'size': None,...",None,None
7,albert/albert-xxlarge-v2,albert,97d3e58863d3a41dc581882f73b34d110b18f1f8,2022-03-02 23:29:04+00:00,2024-02-19 11:02:09+00:00,False,False,None,12152,None,...,fill-mask,None,None,None,None,None,None,"[{'rfilename': '.gitattributes', 'size': None,...",None,None
8,google-bert/bert-base-cased-finetuned-mrpc,google-bert,f150c1d609d1e50dd5e2e5408661cfac8339277c,2022-03-02 23:29:04+00:00,2024-02-19 11:03:21+00:00,False,False,None,52078,None,...,fill-mask,None,None,None,None,None,None,"[{'rfilename': '.gitattributes', 'size': None,...",None,None
9,google-bert/bert-base-cased,google-bert,cd5ef92a9fb2f889e972770a36d4ed042daf221e,2022-03-02 23:29:04+00:00,2024-02-19 11:02:26+00:00,False,False,None,5783879,None,...,fill-mask,None,None,None,None,None,None,"[{'rfilename': '.gitattributes', 'size': None,...",None,None


In [76]:
print(models_df.columns)

Index(['id', 'author', 'sha', 'created_at', 'last_modified', 'private',
       'gated', 'disabled', 'downloads', 'likes', 'library_name', 'tags',
       'pipeline_tag', 'mask_token', 'card_data', 'widget_data', 'model_index',
       'config', 'transformers_info', 'siblings', 'spaces', 'safetensors'],
      dtype='object')


In [77]:
models_df.loc[0]['tags']

['transformers',
 'pytorch',
 'tf',
 'safetensors',
 'albert',
 'fill-mask',
 'exbert',
 'en',
 'dataset:bookcorpus',
 'dataset:wikipedia',
 'arxiv:1909.11942',
 'license:apache-2.0',
 'autotrain_compatible',
 'endpoints_compatible',
 'region:us']

In [78]:
example_model = api.list_models(model_name='meta-llama/Meta-Llama-3.1-8B-Instruct')
example_df = pd.DataFrame(example_model)
example_df.loc[0]['tags']

['transformers',
 'safetensors',
 'llama',
 'text-generation',
 'facebook',
 'meta',
 'pytorch',
 'llama-3',
 'conversational',
 'en',
 'de',
 'fr',
 'it',
 'pt',
 'hi',
 'es',
 'th',
 'arxiv:2204.05149',
 'license:llama3.1',
 'autotrain_compatible',
 'text-generation-inference',
 'endpoints_compatible',
 'region:us']

In [9]:
# Scrape languages from HF

url_languages = 'https://huggingface.co/languages'

response = requests.get(url_languages)
html_content = response.text

soup = BeautifulSoup(html_content, 'html.parser')

code_tags = soup.find_all('code')
tag_language = [code_tag.get_text() for code_tag in code_tags]
# print(tag_language)

tag_language.remove('jax') # 'jax' is the ISO for Jambi Malay (present in 3 datasets, 36 models), impossible to distinguish from JAX the library... TODO: better solution?

In [10]:
def extract_name(full_name):
    pattern = re.compile(r'[^/]+/(.+)')
    match = re.search(pattern, full_name)
    if match:
        return match.group(1) # the part after '/' might also contain version and number of parameters (impossible to extract in a uniform way)
    else:
        return full_name

def match_string(entries, pattern):
    pattern = re.compile(pattern)
    for entry in entries:
        match = pattern.match(entry)
        if match:
            return match.group(1)
    return None

def find_all_matches(entries, pattern):
    pattern = re.compile(pattern)
    matches = []
    for entry in entries:
        match = pattern.match(entry)
        if match:
            matches.append(match.group(1))
    return matches

def match_license(entries):
    return match_string(entries, r'license:(\S+)')

def match_dataset(entries):
    return find_all_matches(entries, r'dataset:(\S+)')

def match_uri(entries):
    uri = match_string(entries, r'arxiv:(\S+)')
    if uri is None:
        uri = match_string(entries, r'doi:(\S+)')
    return uri

In [7]:
def extract_attributes(model_idx):

	model = models_df.loc[model_idx]
	model_tags = models_df.loc[model_idx]['tags']
	model_card_data = None
	try:
		model_card_data = next(api.list_models(model_name=model['id'], full=True, cardData=True)).card_data.to_dict()
	except AttributeError:
		print('No card data available for this model')
	model_attributes = dict()

	model_attributes['name'] = extract_name(model['id'])
	model_attributes['version'] = None # sometimes in model['id'] but impossible to extract in a uniform way
	model_attributes['numberOfParameters'] = None # sometimes in model['id'] or model description but impossible to extract in a uniform way

	model_attributes['quantization'] = None
	for t in model_tags:
		if t in tag_quantization:
			model_attributes['quantization'] = t

	model_attributes['architecture'] = None
	try:
		if model_card_data is not None:
			model_attributes['architecture'] = model_card_data['base_model']
	except KeyError:
		print('No architecture data available for this model')

	model_attributes['languages'] = []
	for t in model_tags:
		if t in tag_language:
			model_attributes['languages'].append(t)

	model_attributes['modelCreator'] = None # TODO: if base_model exists, look for 'author' of the base model
	try:
		if model_card_data is not None:
			base_model = model_card_data['base_model']
			base_model_data = pd.DataFrame(api.list_models(model_name=base_model, full=True))
			model_attributes['modelCreator'] = base_model_data.loc[0]['author']
	except KeyError:
		print('No base model data available for this model')

	model_attributes['licenseToUse'] = match_license(model_tags)

	model_attributes['libraryFramework'] = [] # TODO: change type into list(str) in our model
	for t in model_tags:
		if t in tag_library:
			model_attributes['libraryFramework'].append(t)

	model_attributes['contextLength'] = None
	model_attributes['developers'] = [model['author']]
	model_attributes['openSource'] = True

	model_attributes['uri'] = match_uri(model_tags)

	model_attributes['fineTuned'] = None # if there is a 'base_model' in card_data, it is fine-tuned
	try:
		if model_card_data is not None:
			if 'base_model' in model_card_data:
				model_attributes['fineTuned'] = True
	except KeyError:
		print('No base model data available for this model')

	model_attributes['carbonEmission [CO2eq tons]'] = None
	try:
		if model_card_data is not None:
			model_attributes['carbonEmission [CO2eq tons]'] = model_card_data['co2_eq_emissions']
	except KeyError:
		print('No emission data available for this model')

	model_attributes['tokenizer'] = None

	return model_attributes

In [20]:
def model_to_json(model_idx):
    
    current_path = os.getcwd()
    parent_path = os.path.dirname(current_path)
    result_path = os.path.join(parent_path, 'database', 'hf extracted json')
    
    os.makedirs(result_path, exist_ok=True)
    
    model_attributes = extract_attributes(model_idx)
    
    with open(os.path.join(result_path, 'ChatIMPACT.LargeLanguageModel.json'), 'w', encoding='utf-8') as f:
        json.dump(model_attributes, f, indent=4)

In [21]:
def models_to_json(models_df):
    
    current_path = os.getcwd()
    parent_path = os.path.dirname(current_path)
    result_path = os.path.join(parent_path, 'database', 'hf extracted json')
    
    os.makedirs(result_path, exist_ok=True)

    output = []
    
    for model_idx in range(models_df.shape[0]):
        output.append(extract_attributes(model_idx))
    
    with open(os.path.join(result_path, 'ChatIMPACT.LargeLanguageModel.json'), 'w', encoding='utf-8') as f:
        json.dump(output, f, indent=4)

In [ ]:
models_to_json(models_df)

In [78]:
# Read JSON and count available attributes

def models_statistics():

	architecture_count = 0
	languages_count = 0
	model_creator_count = 0
	license_count = 0
	library_count = 0
	finetuned_count = 0
	carbon_emission_count = 0

	current_path = os.getcwd()
	parent_path = os.path.dirname(current_path)
	result_path = os.path.join(parent_path, 'database', 'hf extracted json')

	models_json = open(os.path.join(result_path, 'ChatIMPACT.LargeLanguageModel.json'))
	models_data_json = json.load(models_json)

	for idx, item in enumerate(models_data_json):
		if item['architecture'] is not None:
			architecture_count += 1
		if item['languages'] is not None:
			languages_count += 1
		if item['modelCreator'] is not None:
			model_creator_count += 1
		if item['licenseToUse'] is not None:
			license_count += 1
		if item['libraryFramework'] is not None:
			library_count += 1
		if item['fineTuned'] is not None:
			finetuned_count += 1
		if item['carbonEmission [CO2eq tons]'] is not None:
			carbon_emission_count += 1
	
	print(f'Number of processed models: {idx + 1}')
	print(f'    Architecture: {architecture_count}')
	print(f'    Languages: {languages_count}')
	print(f'    Model creator: {model_creator_count}')
	print(f'    License to use: {license_count}')
	print(f'    Library: {library_count}')
	print(f'    Fine-tuned: {finetuned_count}')
	print(f'    Carbon emission: {carbon_emission_count}')


In [79]:
models_statistics()

Number of processed models: 1000
    Architecture: 6
    Languages: 1000
    Model creator: 6
    License to use: 172
    Library: 1000
    Fine-tuned: 6
    Carbon emission: 9


In [16]:
# Evaluate availability of attributes

# availability = pd.DataFrame(columns=['id', 'entity name', 'attribute name', 'available API', 'available scraping'])

# llm_attributes = ['name', 'version', 'number of parameters', 'quantization', 'architecture', 'language', 'model creator', 'license to use', 'library framework', 'context length', 'developer', 'open source', 'uri', 'fine-tuned', 'carbon emission', 'tokenizer']
# llm_attributes_API_availability = [True, False, False, False, False, True, True, True, True, False, False, False, True, False, True, False]
# availability['id'] = models_df['id']
# availability['entity name'] = 'LLM'
# availability = availability.loc[availability.index.repeat(len(llm_attributes))].reset_index(drop=True)
# availability['attribute name'] = llm_attributes * len(models_df)
# availability['available API'] = llm_attributes_API_availability * len(models_df)

## Dataset

In [64]:
datasets = api.list_datasets(full=True)

In [65]:
datasets = list(itertools.islice(datasets, 0, 1000))
datasets_df = pd.DataFrame(datasets)
datasets_df.head(10)

,id,author,sha,created_at,last_modified,private,gated,disabled,downloads,downloads_all_time,likes,paperswithcode_id,tags,card_data,siblings
0,amirveyseh/acronym_identification,amirveyseh,15ef643450d589d5883e289ffadeb03563e80a9e,2022-03-02 23:29:22+00:00,2024-01-09 11:39:57+00:00,False,False,False,109,None,19,acronym-identification,"[task_categories:token-classification, annotat...",{},None
1,ade-benchmark-corpus/ade_corpus_v2,ade-benchmark-corpus,4ba01c71687dd7c996597042449448ea312126cf,2022-03-02 23:29:22+00:00,2024-01-09 11:42:58+00:00,False,False,False,285,None,25,None,"[task_categories:text-classification, task_cat...",{},None
2,UCLNLP/adversarial_qa,UCLNLP,c2d5f738db1ad21a4126a144dfbb00cb51e0a4a9,2022-03-02 23:29:22+00:00,2023-12-21 14:20:00+00:00,False,False,False,98,None,32,adversarialqa,"[task_categories:question-answering, task_ids:...",{},None
3,Yale-LILY/aeslc,Yale-LILY,2305f2e63b68056f9b9037a3805c8c196e0d5581,2022-03-02 23:29:22+00:00,2024-01-09 11:49:13+00:00,False,False,False,56,None,12,aeslc,"[task_categories:summarization, annotations_cr...",{},None
4,nwu-ctext/afrikaans_ner_corpus,nwu-ctext,445834a997dce8b40e1d108638064381de80c497,2022-03-02 23:29:22+00:00,2024-01-09 11:51:47+00:00,False,False,False,62,None,6,None,"[task_categories:token-classification, task_id...",{},None
5,fancyzhx/ag_news,fancyzhx,eb185aade064a813bc0b7f42de02595523103ca4,2022-03-02 23:29:22+00:00,2024-03-07 12:02:37+00:00,False,False,False,6718,None,122,ag-news,"[task_categories:text-classification, task_ids...",{},None
6,allenai/ai2_arc,allenai,210d026faf9955653af8916fad021475a3f00453,2022-03-02 23:29:22+00:00,2023-12-21 15:09:48+00:00,False,False,False,614345,None,107,None,"[task_categories:question-answering, task_ids:...",{},None
7,google/air_dialogue,google,dbdbe7bcef8d344bc3c68a05600f3d95917d6898,2022-03-02 23:29:22+00:00,2024-03-07 15:22:15+00:00,False,False,False,43,None,15,None,"[task_categories:text-generation, task_categor...",{},None
8,komari6/ajgt_twitter_ar,komari6,af3f2fa5462ac461b696cb300d66e07ad366057f,2022-03-02 23:29:22+00:00,2024-01-09 11:58:01+00:00,False,False,False,73,None,3,None,"[task_categories:text-classification, task_ids...",{},None
9,legacy-datasets/allegro_reviews,legacy-datasets,71593d1379934286885c53d147bc863ffe830745,2022-03-02 23:29:22+00:00,2024-01-09 11:59:39+00:00,False,False,False,43,None,4,allegro-reviews,"[task_categories:text-classification, task_ids...",{},None


In [98]:
datasets_df.columns

Index(['id', 'author', 'sha', 'created_at', 'last_modified', 'private',
       'gated', 'disabled', 'downloads', 'likes', 'paperswithcode_id', 'tags',
       'card_data', 'siblings'],
      dtype='object')

In [99]:
datasets_df.loc[0]

id                                   amirveyseh/acronym_identification
author                                                      amirveyseh
sha                           15ef643450d589d5883e289ffadeb03563e80a9e
created_at                                   2022-03-02 23:29:22+00:00
last_modified                                2024-01-09 11:39:57+00:00
private                                                          False
gated                                                            False
disabled                                                         False
downloads                                                          120
likes                                                               19
paperswithcode_id                               acronym-identification
tags                 [task_categories:token-classification, annotat...
card_data                                                           {}
siblings                                                          None
Name: 

In [100]:
datasets_df.loc[2]['tags']

['task_categories:question-answering',
 'task_ids:extractive-qa',
 'task_ids:open-domain-qa',
 'annotations_creators:crowdsourced',
 'language_creators:found',
 'multilinguality:monolingual',
 'source_datasets:original',
 'language:en',
 'license:cc-by-sa-4.0',
 'size_categories:10K<n<100K',
 'format:parquet',
 'modality:text',
 'library:datasets',
 'library:pandas',
 'library:mlcroissant',
 'library:polars',
 'arxiv:2002.00293',
 'arxiv:1606.05250',
 'region:us']

In [101]:
def match_language(entries):
    return find_all_matches(entries, r'language:(\S+)')

def match_size(entries):
    return match_string(entries, r'size_categories:(\S+)')

In [111]:
def extract_datasets_attributes(dataset_idx):

	dataset = datasets_df.loc[dataset_idx]
	dataset_tags = datasets_df.loc[dataset_idx]['tags']
	dataset_attributes = dict()

	dataset_attributes['name'] = extract_name(dataset['id'])
	dataset_attributes['size [GB]'] = match_size(dataset_tags) # TODO: how to deal with this?

	dataset_attributes['languages'] = match_language(dataset_tags)

	# dataset_attributes['dataset creator'] = dataset['author'] # TODO: add attribute in our model?

	dataset_attributes['licenseToUse'] = match_license(dataset_tags)

	dataset_attributes['domain'] = []
	for t in dataset_tags:
		if t in tag_domain:
			dataset_attributes['domain'].append(t)

	dataset_attributes['uri'] = match_uri(dataset_tags) # TODO: add multiple URIs when available?

	dataset_attributes['fineTuning'] = None

	return dataset_attributes

In [112]:
def dataset_to_json(datasets_df):
    
    current_path = os.getcwd()
    parent_path = os.path.dirname(current_path)
    result_path = os.path.join(parent_path, 'database', 'hf extracted json')
    
    os.makedirs(result_path, exist_ok=True)

    output = []
    
    for dataset_idx in range(datasets_df.shape[0]):
        output.append(extract_datasets_attributes(dataset_idx))
    
    with open(os.path.join(result_path, 'ChatIMPACT.Dataset.json'), 'w', encoding='utf-8') as f:
        json.dump(output, f, indent=4)

In [113]:
dataset_to_json(datasets_df)

## Downstream Task

In [118]:
# TODO: save converted datasets to JSON in a new directory

def fetch_and_extract_text(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        target_paragraph = soup.find('p', class_='text-[1.2rem] text-gray-500')
        
        if target_paragraph:
            return target_paragraph.get_text().strip()
        else:
            return "Target paragraph not found."
    else:
        return f"Failed to fetch the webpage. Status code: {response.status_code}"

def create_tasks_json():

    current_path = os.getcwd()
    parent_path = os.path.dirname(current_path)
    result_path = os.path.join(parent_path, 'database', 'hf extracted json')
    
    os.makedirs(result_path, exist_ok=True)

    tasks_data = []

    for task in TAG_DOWNSTREAM_TASK:
        url = f"https://huggingface.co/tasks/{task}"
        description = fetch_and_extract_text(url)
        
        tasks_data.append({
            "name": task,
            "description": description, # TODO: text2text generation has no description
            "sub-task": []
        })
        
        print(f"Processed: {task}")
        # time.sleep(0.5)  # Be polite to the server

    with open(result_path + '/ChatIMPACT.DownstreamTask.json', 'w', encoding='utf-8') as f:
        json.dump(tasks_data, f, ensure_ascii=False, indent=2)

In [119]:
create_tasks_json()

Processed: text-classification
Processed: token-classification
Processed: table-question-answering
Processed: question-answering
Processed: zero-shot-classification
Processed: translation
Processed: summarization
Processed: feature-extraction
Processed: text-generation
Processed: fill-mask
Processed: sentence-similarity


## Metrics

In [120]:
# Scrape metrics and descriptions from HF

url_metrics = 'https://huggingface.co/metrics'

response = requests.get(url_metrics)
html_content = response.text

soup = BeautifulSoup(html_content, 'html.parser')

h4_tags = soup.find_all('h4')
metrics = [h4_tag.get_text(strip=True) for h4_tag in h4_tags]
# print(metrics)

p_tags = soup.find_all('p')
descriptions = [p_tag.get_text() for p_tag in p_tags]
descriptions = descriptions[2:] # drop first lines
# print(descriptions)

# remove from the list the metrics withoud description (not useful for our purpose)
metrics.remove('AlhitawiMohammed22/CER_Hu-Evaluation-Metrics')
metrics.remove('Aye10032/loss_metric')
metrics.remove('giulio98/code_eval_outputs')
metrics.remove('maysonma/lingo_judge_metric')
metrics.remove('lvwerra/test')
metrics.remove('sma2023/wil')


assert len(metrics) == len(descriptions)
print(len(metrics))

265


In [121]:
# From the lists, remove the descriptions and then the relative metric in the same index that have in the description 'TODO: add a description here\n\t\t\t\t\t\t'ArithmeticError

for i, description in enumerate(descriptions):
    if 'TODO: add a description here' in description:
        metrics.pop(i)
        descriptions.pop(i)

assert len(metrics) == len(descriptions)
print(len(metrics))

218


In [126]:
def create_metrics_json(metrics, descriptions):

    metrics_data = []

    current_path = os.getcwd()
    parent_path = os.path.dirname(current_path)
    result_path = os.path.join(parent_path, 'database', 'hf extracted json')
    
    os.makedirs(result_path, exist_ok=True)
    
    for idx in range(len(metrics)):
        metric_attributes = dict()

        metric_attributes['name'] = metrics[idx]
        metric_attributes['description'] = descriptions[idx]
        metric_attributes['context'] = None
        metric_attributes['featureBased/endToEnd'] = None
        metric_attributes['granularity'] = None

    
        metrics_data.append(metric_attributes)
        

    with open(os.path.join(result_path, 'ChatIMPACT.Metric.json'), 'w', encoding='utf-8') as f:
        json.dump(metrics_data, f, indent=4)

In [127]:
create_metrics_json(metrics, descriptions)

## Train relationship

In [54]:
def extract_train_relationship():
    
    train_relationship_list = []
    
    for model_idx in range(len(models_df)):
        
        model = models_df.loc[model_idx]
        model_tags = models_df.loc[model_idx]['tags']
        datasets = match_dataset(model_tags)
        
        if not datasets:
            continue
        else:
            train_relationship = dict()
            train_relationship['Models'] = extract_name(model['id']) # {"$oid": <...>} for MongoDB
            train_relationship['Datasets'] = datasets # [{"$oid": <...>}, ..., {"$oid": <...>}] for MongoDB
            train_relationship_list.append(train_relationship)
    
    return train_relationship_list

In [55]:
def create_train_relationship_json():
	
	current_path = os.getcwd()
	parent_path = os.path.dirname(current_path)
	result_path = os.path.join(parent_path, 'database', 'hf extracted json')
	
	os.makedirs(result_path, exist_ok=True)

	train_relationship = extract_train_relationship()

	with open(os.path.join(result_path, 'ChatIMPACT.TrainRelationship.json'), 'w', encoding='utf-8') as f:
		json.dump(train_relationship, f, indent=4)

In [56]:
create_train_relationship_json()

## SuitedFor relationship

In [57]:
def extract_suited_for_relationship():
    
    suited_for_relationship_list = []
    
    for model_idx in range(len(models_df)):
        
        model = models_df.loc[model_idx]
        model_tags = models_df.loc[model_idx]['tags']
        
        tasks = []
        for t in model_tags:
            if t in TAG_DOWNSTREAM_TASK:
                tasks.append(t)

        if not tasks:
            continue
        else:
            suited_for_relationship = dict()
            suited_for_relationship['LargeLanguageModel'] = extract_name(model['id']) # {"$oid": <...>} for MongoDB
            suited_for_relationship['DownstreamTask'] = tasks # [{"$oid": <...>}, ..., {"$oid": <...>}] for MongoDB
            suited_for_relationship_list.append(suited_for_relationship)
    
    return suited_for_relationship_list

In [58]:
def create_suited_for_relationship_json():
	
	current_path = os.getcwd()
	parent_path = os.path.dirname(current_path)
	result_path = os.path.join(parent_path, 'database', 'hf extracted json')
	
	os.makedirs(result_path, exist_ok=True)

	suited_for_relationship = extract_suited_for_relationship()

	with open(os.path.join(result_path, 'ChatIMPACT.SuitedForRelationship.json'), 'w', encoding='utf-8') as f:
		json.dump(suited_for_relationship, f, indent=4)

In [59]:
create_suited_for_relationship_json()

# Enable relationship

In [60]:
def match_tasks(entries):
    return find_all_matches(entries, r'task_categories:(\S+)')

In [61]:
def extract_enable_relationship():
	
	enable_relationship_list = []
	
	for dataset_idx in range(len(datasets_df)):
		
		dataset = datasets_df.loc[dataset_idx]
		dataset_tags = datasets_df.loc[dataset_idx]['tags']
		
		tasks = match_tasks(dataset_tags)

		if not tasks:
			continue
		else:
			enable_relationship = dict()
			enable_relationship['Dataset'] = extract_name(dataset['id']) # {"$oid": <...>} for MongoDB
			enable_relationship['DownstreamTask'] = tasks # [{"$oid": <...>}, ..., {"$oid": <...>}] for MongoDB
			enable_relationship_list.append(enable_relationship)
	
	return enable_relationship_list

In [62]:
def create_enable_relationship_json():
	
	current_path = os.getcwd()
	parent_path = os.path.dirname(current_path)
	result_path = os.path.join(parent_path, 'database', 'hf extracted json')
	
	os.makedirs(result_path, exist_ok=True)

	enable_relationship = extract_enable_relationship()

	with open(os.path.join(result_path, 'ChatIMPACT.EnableRelationship.json'), 'w', encoding='utf-8') as f:
		json.dump(enable_relationship, f, indent=4)

In [66]:
create_enable_relationship_json()

# Assess relationship

In [ ]:
# TODO: here https://huggingface.co/tasks some tasks have associated metrics, we could scrape the tasks one by one

# Evaluate relationship

In [70]:
# TODO: check that this is correct (the output and the model cards on HF do not seem to be coherent?)
# Model card template: https://github.com/huggingface/hub-docs/blob/main/modelcard.md?plain=1

def extract_evaluate_relationship():

	evaluate_relationship_list = []

	for model_idx in range(len(models_df)):

		model = models_df.loc[model_idx]
		
		try:
			model_card_data = next(api.list_models(model_name=model['id'], full=True, cardData=True)).card_data.to_dict()
		except AttributeError:
			print('No card data available for this model')
		
		metrics = []
		if model_card_data is not None:
			if 'metrics' in model_card_data:
				metrics = model_card_data['metrics']
			
		if not metrics:
			continue
		else:
			evaluate_relationship = dict()
			evaluate_relationship['LargeLanguageModel'] = extract_name(model['id'])
			evaluate_relationship['Metric'] = metrics
			evaluate_relationship_list.append(evaluate_relationship)
	
	return evaluate_relationship_list

In [71]:
def create_evaluate_relationship_json():
	
	current_path = os.getcwd()
	parent_path = os.path.dirname(current_path)
	result_path = os.path.join(parent_path, 'database', 'hf extracted json')
	
	os.makedirs(result_path, exist_ok=True)

	evaluate_relationship = extract_evaluate_relationship()

	with open(os.path.join(result_path, 'ChatIMPACT.EvaluateRelationship.json'), 'w', encoding='utf-8') as f:
		json.dump(evaluate_relationship, f, indent=4)

In [ ]:
create_evaluate_relationship_json()